# 컴퓨터 비전 기초 학습

이 노트북은 컴퓨터 비전의 기본 개념들을 인터랙티브하게 학습할 수 있도록 구성되었습니다.

## 목차
1. [환경 설정](#환경-설정)
2. [이미지 기초](#이미지-기초)
3. [이미지 필터링](#이미지-필터링)
4. [임계값 처리](#임계값-처리)
5. [실습 프로젝트](#실습-프로젝트)

## 환경 설정

필요한 라이브러리들을 import합니다.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

# 한글 폰트 설정 (matplotlib)
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

# 이미지 표시 함수
def show_images(images, titles=None, figsize=(15, 5)):
    """여러 이미지를 나란히 표시"""
    n = len(images)
    fig, axes = plt.subplots(1, n, figsize=figsize)
    
    if n == 1:
        axes = [axes]
    
    for i, img in enumerate(images):
        if len(img.shape) == 3:
            axes[i].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        else:
            axes[i].imshow(img, cmap='gray')
        
        if titles:
            axes[i].set_title(titles[i])
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

print("환경 설정 완료!")
print(f"OpenCV 버전: {cv2.__version__}")
print(f"NumPy 버전: {np.__version__}")

## 이미지 기초

### 샘플 이미지 생성

실습을 위한 다양한 샘플 이미지를 생성해보겠습니다.

In [ ]:
def create_sample_images():
    """다양한 샘플 이미지 생성"""
    
    # 1. 체스보드 패턴
    checkerboard = np.zeros((200, 200), dtype=np.uint8)
    square_size = 25
    for i in range(0, 200, square_size):
        for j in range(0, 200, square_size):
            if (i // square_size + j // square_size) % 2 == 0:
                checkerboard[i:i+square_size, j:j+square_size] = 255
    
    # 2. 원형 패턴
    circles = np.zeros((200, 200), dtype=np.uint8)
    cv2.circle(circles, (50, 100), 30, 255, -1)
    cv2.circle(circles, (150, 100), 25, 128, -1)
    cv2.circle(circles, (100, 150), 35, 200, -1)
    
    # 3. 그라디언트
    gradient = np.linspace(0, 255, 200, dtype=np.uint8)
    gradient = np.tile(gradient, (200, 1))
    
    return checkerboard, circles, gradient

# 샘플 이미지 생성 및 표시
checkerboard, circles, gradient = create_sample_images()

show_images([checkerboard, circles, gradient], 
           ['체스보드', '원형 패턴', '그라디언트'])

# 이미지 속성 확인
print(f"체스보드 이미지:")
print(f"  크기: {checkerboard.shape}")
print(f"  데이터 타입: {checkerboard.dtype}")
print(f"  평균 밝기: {np.mean(checkerboard):.1f}")
print(f"  최솟값: {np.min(checkerboard)}, 최댓값: {np.max(checkerboard)}")

### 노이즈 추가

실제 이미지에는 다양한 종류의 노이즈가 포함됩니다. 노이즈를 추가해보고 그 영향을 관찰해보겠습니다.

In [ ]:
def add_noise(image, noise_type='gaussian', intensity=25):
    """이미지에 노이즈 추가"""
    
    if noise_type == 'gaussian':
        noise = np.random.normal(0, intensity, image.shape)
        noisy = image.astype(np.float32) + noise
        
    elif noise_type == 'salt_pepper':
        noisy = image.copy().astype(np.float32)
        num_salt = int(intensity * image.size / 1000)
        
        # Salt noise
        coords = [np.random.randint(0, i-1, num_salt) for i in image.shape]
        noisy[tuple(coords)] = 255
        
        # Pepper noise
        coords = [np.random.randint(0, i-1, num_salt) for i in image.shape]
        noisy[tuple(coords)] = 0
    
    return np.clip(noisy, 0, 255).astype(np.uint8)

# 다양한 노이즈 추가
original = checkerboard
gaussian_noisy = add_noise(original, 'gaussian', 30)
salt_pepper_noisy = add_noise(original, 'salt_pepper', 20)

show_images([original, gaussian_noisy, salt_pepper_noisy], 
           ['원본', '가우시안 노이즈', '잡점 노이즈'])

# 노이즈의 영향 분석
print("노이즈 영향 분석:")
print(f"원본 표준편차: {np.std(original):.2f}")
print(f"가우시안 노이즈 후: {np.std(gaussian_noisy):.2f}")
print(f"잡점 노이즈 후: {np.std(salt_pepper_noisy):.2f}")

## 이미지 필터링

### 블러 필터

가우시안 블러는 노이즈 제거에 효과적입니다.

In [ ]:
# 가우시안 블러 적용
blur_3x3 = cv2.GaussianBlur(gaussian_noisy, (3, 3), 0)
blur_7x7 = cv2.GaussianBlur(gaussian_noisy, (7, 7), 0)
blur_15x15 = cv2.GaussianBlur(gaussian_noisy, (15, 15), 0)

show_images([gaussian_noisy, blur_3x3, blur_7x7, blur_15x15], 
           ['노이즈 이미지', '3x3 블러', '7x7 블러', '15x15 블러'], 
           figsize=(20, 5))

# 노이즈 감소 효과 측정
def calculate_psnr(original, processed):
    mse = np.mean((original.astype(np.float32) - processed.astype(np.float32)) ** 2)
    if mse == 0:
        return float('inf')
    return 20 * np.log10(255.0 / np.sqrt(mse))

print("PSNR (높을수록 좋음):")
print(f"3x3 블러: {calculate_psnr(original, blur_3x3):.2f} dB")
print(f"7x7 블러: {calculate_psnr(original, blur_7x7):.2f} dB")
print(f"15x15 블러: {calculate_psnr(original, blur_15x15):.2f} dB")

### 양방향 필터

양방향 필터는 에지를 보존하면서 노이즈를 제거합니다.

In [ ]:
# 양방향 필터 vs 가우시안 블러 비교
gaussian_blur = cv2.GaussianBlur(gaussian_noisy, (9, 9), 0)
bilateral = cv2.bilateralFilter(gaussian_noisy, 9, 75, 75)

show_images([gaussian_noisy, gaussian_blur, bilateral], 
           ['노이즈 이미지', '가우시안 블러', '양방향 필터'])

# 에지 보존 효과 측정
def edge_strength(image):
    laplacian = cv2.Laplacian(image, cv2.CV_64F)
    return np.var(laplacian)

print("에지 강도 (높을수록 선명):")
print(f"원본: {edge_strength(original):.2f}")
print(f"가우시안 블러: {edge_strength(gaussian_blur):.2f}")
print(f"양방향 필터: {edge_strength(bilateral):.2f}")

### 미디안 필터

미디안 필터는 잡점 노이즈(salt & pepper noise) 제거에 매우 효과적입니다.

In [ ]:
# 미디안 필터 적용
median_3 = cv2.medianBlur(salt_pepper_noisy, 3)
median_5 = cv2.medianBlur(salt_pepper_noisy, 5)
gaussian_on_sp = cv2.GaussianBlur(salt_pepper_noisy, (5, 5), 0)

show_images([salt_pepper_noisy, median_3, median_5, gaussian_on_sp], 
           ['잡점 노이즈', '미디안 3x3', '미디안 5x5', '가우시안 5x5'], 
           figsize=(20, 5))

print("잡점 노이즈 제거 효과:")
print(f"미디안 3x3 PSNR: {calculate_psnr(original, median_3):.2f} dB")
print(f"미디안 5x5 PSNR: {calculate_psnr(original, median_5):.2f} dB")
print(f"가우시안 PSNR: {calculate_psnr(original, gaussian_on_sp):.2f} dB")

## 임계값 처리

### 기본 임계값 처리

In [ ]:
# 다양한 임계값으로 이진화
test_img = circles

_, thresh_64 = cv2.threshold(test_img, 64, 255, cv2.THRESH_BINARY)
_, thresh_128 = cv2.threshold(test_img, 128, 255, cv2.THRESH_BINARY)
_, thresh_192 = cv2.threshold(test_img, 192, 255, cv2.THRESH_BINARY)

show_images([test_img, thresh_64, thresh_128, thresh_192], 
           ['원본', '임계값 64', '임계값 128', '임계값 192'], 
           figsize=(20, 5))

# 각 임계값의 전경 비율
for thresh, binary in [(64, thresh_64), (128, thresh_128), (192, thresh_192)]:
    ratio = np.sum(binary == 255) / binary.size * 100
    print(f"임계값 {thresh}: 전경 비율 {ratio:.1f}%")

### Otsu 자동 임계값

In [ ]:
# Otsu 방법으로 자동 임계값 결정
otsu_thresh, otsu_binary = cv2.threshold(test_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# 히스토그램과 함께 표시
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 원본 이미지
axes[0].imshow(test_img, cmap='gray')
axes[0].set_title('원본 이미지')
axes[0].axis('off')

# 히스토그램
axes[1].hist(test_img.ravel(), bins=256, range=[0, 256], alpha=0.7)
axes[1].axvline(otsu_thresh, color='red', linestyle='--', label=f'Otsu 임계값: {otsu_thresh:.0f}')
axes[1].set_title('히스토그램')
axes[1].set_xlabel('픽셀 값')
axes[1].set_ylabel('빈도')
axes[1].legend()

# 이진화 결과
axes[2].imshow(otsu_binary, cmap='gray')
axes[2].set_title('Otsu 이진화 결과')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print(f"Otsu가 선택한 최적 임계값: {otsu_thresh:.0f}")

### 적응적 임계값 처리

In [ ]:
# 조명이 불균등한 이미지 생성
uneven_img = np.zeros((200, 200), dtype=np.uint8)

# 그라디언트 배경
for i in range(200):
    for j in range(200):
        uneven_img[i, j] = int(50 + 150 * i / 200)

# 객체 추가
cv2.rectangle(uneven_img, (50, 50), (100, 100), 255, -1)
cv2.rectangle(uneven_img, (120, 120), (170, 170), 255, -1)

# 다양한 임계값 방법 적용
_, simple_thresh = cv2.threshold(uneven_img, 128, 255, cv2.THRESH_BINARY)
adaptive_mean = cv2.adaptiveThreshold(uneven_img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, 
                                     cv2.THRESH_BINARY, 11, 2)
adaptive_gaussian = cv2.adaptiveThreshold(uneven_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                        cv2.THRESH_BINARY, 11, 2)

show_images([uneven_img, simple_thresh, adaptive_mean, adaptive_gaussian], 
           ['불균등 조명', '단순 임계값', '적응적(평균)', '적응적(가우시안)'], 
           figsize=(20, 5))

print("조명이 불균등한 이미지에서는 적응적 임계값이 더 효과적입니다!")

## 실습 프로젝트

### 이미지 개선 파이프라인

지금까지 학습한 기법들을 조합하여 이미지를 개선해보겠습니다.

In [ ]:
# 복합 노이즈가 있는 이미지 생성
degraded_img = checkerboard.copy()

# 가우시안 노이즈 추가
degraded_img = add_noise(degraded_img, 'gaussian', 20)

# 잡점 노이즈 추가
degraded_img = add_noise(degraded_img, 'salt_pepper', 10)

# 어둡게 만들기
degraded_img = (degraded_img * 0.7).astype(np.uint8)

# 개선 파이프라인 적용
# 1단계: 잡점 노이즈 제거
step1 = cv2.medianBlur(degraded_img, 3)

# 2단계: 가우시안 노이즈 제거 (에지 보존)
step2 = cv2.bilateralFilter(step1, 5, 50, 50)

# 3단계: 대비 개선 (히스토그램 균등화)
step3 = cv2.equalizeHist(step2)

# 4단계: 선명도 향상
blurred = cv2.GaussianBlur(step3, (0, 0), 1.0)
step4 = cv2.addWeighted(step3, 1.5, blurred, -0.5, 0)

# 결과 비교
show_images([checkerboard, degraded_img, step1, step2, step3, step4], 
           ['원본', '열화된 이미지', '1단계', '2단계', '3단계', '최종 결과'], 
           figsize=(24, 4))

# 품질 측정
print("이미지 품질 개선 결과:")
print(f"열화된 이미지 PSNR: {calculate_psnr(checkerboard, degraded_img):.2f} dB")
print(f"최종 결과 PSNR: {calculate_psnr(checkerboard, step4):.2f} dB")
print(f"PSNR 개선: {calculate_psnr(checkerboard, step4) - calculate_psnr(checkerboard, degraded_img):.2f} dB")

## 결론

이 노트북에서 다음과 같은 내용을 학습했습니다:

1. **이미지 기초**: NumPy 배열로서의 이미지, 다양한 노이즈 유형
2. **필터링 기법**: 가우시안 블러, 양방향 필터, 미디안 필터
3. **임계값 처리**: 고정 임계값, Otsu 방법, 적응적 임계값
4. **실습 프로젝트**: 여러 기법을 조합한 이미지 개선 파이프라인

### 다음 단계

- 실제 사진 이미지로 실험해보기
- 더 고급 필터링 기법 학습 (Non-local means, BM3D 등)
- 객체 검출 및 특징 추출 기법 학습
- 딥러닝 기반 이미지 처리 방법 탐구